# IDAES Skeleton Unit Model

This notebook demonstrates usage of the IDAES Skeleton Unit Model, which provides a generic "bare bones" unit for user-defined models and custom variable and constraint sets. To allow maximum versatility, this unit may be defined as a surrogate model or a custom equation-oriented model. Users must add ports and variables that match connected models, and this is facilitated through a provided method to add port-variable sets.

For users who wish to train surrogates with IDAES tools and insert obtained models into a flowsheet, see more detailed information on [IDAES Surrogate Tools](https://idaes-pse.readthedocs.io/en/stable/explanations/modeling_extensions/surrogate/index.html).

# 1. Motivation

In many cases, a specific application requires a unique unit operation that does not exist in the IDAES repository. Custom user models may source from external scripts, import surrogate equations or use first-principles calculations. However, IDAES flowsheets adhere to a standardized modeling hierarchy and simple Pyomo models do not always follow these conventions. Additionally, simple flowsheet submodels often require integration with other IDAES unit models which requires consistency between corresponding port variables, stream properties and physical unit sets, as well as proper usage of `ControlVolume` blocks.

The IDAES `SkeletonUnitModel` allows custom creation of user models blocks that do not require `ControlVolume` blocks, and enabling connection with standard IDAES unit models that do contain `ControlVolume` blocks. To motivate the usefulness and versatility of this tool, we will consider a simple pervaporation unit. The custom model does not require rigourous thermodynamic calculations contained in adjacent unit models, and using a Skeleton model allows definition of only required variables and constraints. The new block does require state variable connections for the inlet and outlet streams. We will demonstrate this scenario below to highlight the usage and benefits of the Skeleton model.

# 2. Example - Pervaporation

Pervaporation is a low-energy separation process, and is particularly advantageous over distillation for azeotropic solutions or aqueous mixtures of heavy alcohols. Ethylene glycol is more environmentally friendly than typical chloride- and bromide-based dessicants, and is a common choice for commericial recovery of water from flue gas via liquid spray columns. Due to ethylene glycol's high boiling point, diffusion-based water recovery is economically favorable compared to distillation-based processes. The following example and flux correlation are taken from the literature source below:

Jennifer Runhong Du, Amit Chakma, X. Feng, Dehydration of ethylene glycol by pervaporation using poly(N,N-dimethylaminoethyl methacrylate)/polysulfone composite membranes, Separation and Purification Technology, Volume 64, Issue 1, 2008, Pages 63-70, ISSN 1383-5866, https://doi.org/10.1016/j.seppur.2008.08.004.

The process is adapted from the literature, utilizing an inlet aqueous glycol feed circulated through a feed tank-membrane-feed tank recycle loop while permeate is continuously extracted by the membrane. To demonstrate the usefulness of the Skeleton model, we will model this system as a Mixer and custom Pervaporation unit per the diagram below and define the flux as an empirical custom mass balance term rather than requiring rigorous diffusion calculations. We will also circumvent the need for a vapor phase and VLE calculations by manually calculating the duty to condense and collect permeate vapor, and use correlations for steady-state fluxes to avoid a recycle requiring tear calculations.

![](pervaporation_process.png)

## 2.1 Pyomo and IDAES Imports
We will begin with relevant imports. We will need basic Pyomo and IDAES components:

In [ ]:
import pytest
from pyomo.environ import (check_optimal_termination,
                           ConcreteModel,
                           Constraint,
                           Expression,
                           Objective,
                           maximize,
                           Var,
                           Set,
                           TransformationFactory,
                           value,
                           exp,
                           units as pyunits)
from pyomo.network import Arc
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed, SkeletonUnitModel, Mixer, Product
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.core.solvers import get_solver
from pyomo.util.check_units import assert_units_consistent

# import thermophysical properties
import eg_h2o_ideal as thermo_props
from idaes.models.properties.modular_properties import GenericParameterBlock
from idaes.core.util.constants import Constants

## 2.2 Build Flowsheet

We will build a simple model manually defining state variables relations entering and exiting the pervaporation unit. As shown below, we may define our pre-separation mixer as usual:

In [ ]:
# build the flowsheet
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

m.fs.thermo_params = GenericParameterBlock(**thermo_props.config_dict)

m.fs.WATER = Feed(property_package=m.fs.thermo_params)
m.fs.GLYCOL = Feed(property_package=m.fs.thermo_params)

m.fs.M101 = Mixer(property_package=m.fs.thermo_params,
                  inlet_list=["water_feed", "glycol_feed"])

m.fs.RETENTATE = Product(property_package=m.fs.thermo_params)
m.fs.PERMEATE = Product(property_package=m.fs.thermo_params)

## 2.2 Defining Skeleton Model and Connections

Now that our flowsheet exists, we can manually define variables, units, constraints and ports for our custom pervaporation unit model. By using a Skeleton model, we avoid rigorous mass and energy balances and phase equilibrium which impact model tractability. Instead, we define state variable relations as below - note that we include the fluxes as outlet flow terms. In this model, the variables specify an `FpcTP` system where molar flow of each component, temperature and pressure are selected as state variables:

In [ ]:
# define Skeleton model for pervaporation unit
m.fs.pervap = SkeletonUnitModel(dynamic=False)
m.fs.pervap.comp_list = Set(initialize=["water", "ethylene_glycol"])
m.fs.pervap.phase_list = Set(initialize=["Liq"])

# input vars for skeleton
# m.fs.time is a pre-initialized Set belonging to the FlowsheetBlock; for dynamic=False, time=[0]
m.fs.pervap.flow_in = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                     initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.pervap.temperature_in = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.pervap.pressure_in = Var(m.fs.time, initialize=101e3, units=pyunits.Pa)

# output vars for skeleton
m.fs.pervap.perm_flow = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                      initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.pervap.ret_flow = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                      initialize=1.0, units=pyunits.mol/pyunits.s)
m.fs.pervap.temperature_out = Var(m.fs.time, initialize=298.15, units=pyunits.K)
m.fs.pervap.pressure_out = Var(m.fs.time, initialize=101e3, units=pyunits.Pa)
m.fs.pervap.vacuum = Var(m.fs.time, initialize=1.3e3, units=pyunits.Pa)

# dictionaries relating state properties to custom variables
inlet_dict = {"flow_mol_phase_comp": m.fs.pervap.flow_in,
              "temperature": m.fs.pervap.temperature_in,
              "pressure": m.fs.pervap.pressure_in}
retentate_dict = {"flow_mol_phase_comp": m.fs.pervap.ret_flow,
                  "temperature": m.fs.pervap.temperature_out,
                  "pressure": m.fs.pervap.pressure_out}
permeate_dict = {"flow_mol_phase_comp": m.fs.pervap.perm_flow,
                 "temperature": m.fs.pervap.temperature_out,
                 "pressure": m.fs.pervap.vacuum}

m.fs.pervap.add_ports(name="inlet", member_dict=inlet_dict)
m.fs.pervap.add_ports(name="retentate", member_dict=retentate_dict)
m.fs.pervap.add_ports(name="permeate", member_dict=permeate_dict)

# internal vars for skeleton
energy_activation_dict = {(0, 'Liq', 'water'): 51e3, (0, 'Liq', 'ethylene_glycol'): 53e3}
m.fs.pervap.energy_activation = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                    initialize=energy_activation_dict, units=pyunits.J/pyunits.mol)
m.fs.pervap.energy_activation.fix()

permeance_dict = {(0, 'Liq', 'water'): 5611320, (0, 'Liq', 'ethylene_glycol'): 22358.88}  # calculated from literature data
m.fs.pervap.permeance = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                            initialize=permeance_dict, units=pyunits.mol/pyunits.s/pyunits.m**2)
m.fs.pervap.permeance.fix()

m.fs.pervap.area = Var(m.fs.time, initialize=6, units = pyunits.m**2)
m.fs.pervap.area.fix()

latent_heat_dict = {(0, 'Liq', 'water'): 40.660e3, (0, 'Liq', 'ethylene_glycol'): 56.9e3}
m.fs.pervap.latent_heat_of_vaporization = Var(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list,
                                              initialize=latent_heat_dict, units=pyunits.J/pyunits.mol)
m.fs.pervap.latent_heat_of_vaporization.fix()
m.fs.pervap.heat_duty = Var(m.fs.time, initialize=1, units=pyunits.J/pyunits.s)  # we will calculate this later

Let's define our surrogate equations for flux and permeance, and link them to the port variables. Users can use this structure to write custom relations between inlet and outlet streams; for example, here we define the outlet flow of the pervaporation unit as a sum of the inlet flow and calculated recovery fluxes. By defining model constraints in lieu of rigorous mass balances, we add the flux as a custom mass balance term via an empirical correlation and calculate only the condensation duty rather than implementing full energy balance calculations:

In [ ]:
# Surrogate and first principles model equations

# flux equation (gas constant is defined as J/mol-K)

def rule_permeate_flux(pervap, t, p, i):
    return pervap.permeate.flow_mol_phase_comp[t, p, i] / pervap.area[t] == (pervap.permeance[t, p, i] *
                                                                             exp(-pervap.energy_activation[t, p, i] /
                                                                                 (Constants.gas_constant *
                                                                                  pervap.inlet.temperature[t]))
                                                                            )
m.fs.pervap.eq_permeate_flux = Constraint(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list, rule=rule_permeate_flux)

# permeate condensation equation
# heat duty based on condensing all of permeate product vapor
# avoids the need for a Heater or HeatExchanger unit model

def rule_duty(pervap, t):
    return pervap.heat_duty[t] == sum(pervap.latent_heat_of_vaporization[t, p, i] * pervap.permeate.flow_mol_phase_comp[t, p, i]
                                      for p in pervap.phase_list for i in pervap.comp_list)
m.fs.pervap.eq_duty = Constraint(m.fs.time, rule=rule_duty) 

# flow equation adding total recovery as a custom mass balance term
def rule_retentate_flow(pervap, t, p, i):
    return pervap.retentate.flow_mol_phase_comp[t, p, i] == (pervap.inlet.flow_mol_phase_comp[t, p, i] -
                                                            pervap.permeate.flow_mol_phase_comp[t, p, i])
m.fs.pervap.eq_retentate_flow = Constraint(m.fs.time, m.fs.pervap.phase_list, m.fs.pervap.comp_list, rule=rule_retentate_flow)

Finally, let's define the Arc connecting our two models (IDAES Mixer and custom Pervaporation) and build the flowsheet network:

In [ ]:
m.fs.s01 = Arc(source=m.fs.WATER.outlet, destination=m.fs.M101.water_feed)
m.fs.s02 = Arc(source=m.fs.GLYCOL.outlet, destination=m.fs.M101.glycol_feed)
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.pervap.inlet)
m.fs.s04 = Arc(source=m.fs.pervap.permeate, destination=m.fs.PERMEATE.inlet)
m.fs.s05 = Arc(source=m.fs.pervap.retentate, destination=m.fs.RETENTATE.inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

Let's see how many degrees of freedom the flowsheet has:

In [ ]:
print(degrees_of_freedom(m))

## 2.3 Inlet Specifications

To obtain a square problem with zero degrees of freedom, we specify the inlet water flow, ethylene glycol flow, temperature and pressure for each feed stream, as well as the permeate stream pressure:

In [ ]:
m.fs.WATER.outlet.flow_mol_phase_comp[0, "Liq", "water"].fix(0.34)  # mol/s
m.fs.WATER.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(1e-6)  # mol/s
m.fs.WATER.outlet.temperature.fix(318.15)  # K
m.fs.WATER.outlet.pressure.fix(101.325e3)  # Pa

m.fs.GLYCOL.outlet.flow_mol_phase_comp[0, "Liq", "water"].fix(1e-6)  # mol/s
m.fs.GLYCOL.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].fix(0.66)  # mol/s
m.fs.GLYCOL.outlet.temperature.fix(318.15)  # K
m.fs.GLYCOL.outlet.pressure.fix(101.325e3)  # Pa

Additionally, we need to pass rules defining the temperature and pressure outlets of the pervaporation unit:

In [ ]:
# Add a constraint to calculate the outlet temperature. 
# Here, assume outlet temperature is the same as inlet temperature for illustration
# in reality, temperature change from latent heat loss through membrane is negligible

def rule_temp_out(pervap, t):
    return pervap.inlet.temperature[t] == pervap.retentate.temperature[t]
m.fs.pervap.temperature_out_calculation = Constraint(m.fs.time, rule=rule_temp_out) 

# Add a constraint to calculate the retentate pressure
# Here, assume the retentate pressure is the same as the inlet pressure for illustration
# in reality, pressure change from mass loss through membrane is negligible

def rule_pres_out(pervap, t):
    return pervap.inlet.pressure[t] == pervap.retentate.pressure[t]
m.fs.pervap.pressure_out_calculation = Constraint(m.fs.time, rule=rule_pres_out)

# fix permeate vacuum pressure
m.fs.PERMEATE.inlet.pressure.fix(1.3e3)

assert degrees_of_freedom(m) == 0

## 2.4 Custom Initialization
In addition to allowing custom variable and constraint defintions, the Skeleton model enables implementation of a custom initialization scheme. Complex unit operations may present unique tractability issues, and users have precise control over piecewise unit model solving.

In [ ]:
# Add this to the imports
from pyomo.util.calc_var_value import calculate_variable_from_constraint

def my_initialize(unit, **kwargs):
    # Callback for user provided initialization sequence
    # Fix the inlet state
    unit.inlet.flow_mol_phase_comp.fix()
    unit.inlet.pressure.fix() 
    unit.inlet.temperature.fix()

    # Calculate the values of the remaining variables
    for t in m.fs.time: 

        calculate_variable_from_constraint(
            unit.permeate.flow_mol_phase_comp[t, "Liq", "water"],
            unit.eq_permeate_flux[t, "Liq", "water"])

        calculate_variable_from_constraint(
            unit.permeate.flow_mol_phase_comp[t, "Liq", "ethylene_glycol"],
            unit.eq_permeate_flux[t, "Liq", "ethylene_glycol"])

        calculate_variable_from_constraint(
            unit.heat_duty[t],
            unit.eq_duty[t])

        calculate_variable_from_constraint(
            unit.retentate.flow_mol_phase_comp[t, "Liq", "water"],
            unit.eq_retentate_flow[t, "Liq", "water"])

        calculate_variable_from_constraint(
            unit.retentate.flow_mol_phase_comp[t, "Liq", "ethylene_glycol"],
            unit.eq_retentate_flow[t, "Liq", "ethylene_glycol"])

        calculate_variable_from_constraint(
            unit.retentate.temperature[t],
            unit.temperature_out_calculation[t])

        calculate_variable_from_constraint(
            unit.retentate.pressure[t],
            unit.pressure_out_calculation[t])

    assert degrees_of_freedom(unit) == 0
    if degrees_of_freedom(unit) == 0:
        res = solver.solve(unit, tee = True)
    unit.inlet.flow_mol_phase_comp.unfix()
    unit.inlet.temperature.unfix()
    unit.inlet.pressure.unfix()
    print('Custom initialization routine complete: ', res.solver.message)

solver = get_solver()

m.fs.WATER.initialize()
propagate_state(m.fs.s01)

m.fs.GLYCOL.initialize()
propagate_state(m.fs.s02)

m.fs.pervap.config.initializer = my_initialize
my_initialize(m.fs.pervap)
propagate_state(m.fs.s03)

m.fs.PERMEATE.initialize()
propagate_state(m.fs.s04)

m.fs.RETENTATE.initialize()

results = solver.solve(m, tee = True)

Let's check the results:

In [ ]:
# print results

m.fs.WATER.report()
m.fs.GLYCOL.report()
m.fs.PERMEATE.report()
m.fs.RETENTATE.report()

In [ ]:
# separation factor for results analysis
m.fs.inlet_water_frac = Expression(expr=(m.fs.pervap.inlet.flow_mol_phase_comp[0, "Liq", "water"] /
                                         sum(m.fs.pervap.inlet.flow_mol_phase_comp[0, "Liq", i]
                                             for i in m.fs.pervap.comp_list)
                                        ))
m.fs.permeate_water_frac = Expression(expr=(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", "water"] /
                                            sum(m.fs.pervap.permeate.flow_mol_phase_comp[0, "Liq", i]
                                                for i in m.fs.pervap.comp_list)
                                           ))
m.fs.separation_factor = Expression(expr=(m.fs.permeate_water_frac/(1-m.fs.permeate_water_frac)) /
                                    (m.fs.inlet_water_frac/(1-m.fs.inlet_water_frac)))

print(f"Inlet water mole fraction: {value(m.fs.inlet_water_frac)}")
print(f"Permeate water mole fraction: {value(m.fs.permeate_water_frac)}")
print(f"Separation factor: {value(m.fs.separation_factor)}")
print(f"Condensation duty: {value(m.fs.pervap.heat_duty[0]/1000)} kW")
print(f"Duty per mole water recovered: {value(m.fs.pervap.heat_duty[0]/(1000*m.fs.PERMEATE.inlet.flow_mol_phase_comp[0, 'Liq', 'water']*3600))} kW-h / mol")

In [ ]:
# check results
assert check_optimal_termination(results)
assert_units_consistent(m)

assert value(m.fs.PERMEATE.inlet.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.1426, rel=1e-3)
assert value(m.fs.PERMEATE.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.0002667, rel=1e-3)
assert value(m.fs.RETENTATE.inlet.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.1974, rel=1e-3)
assert value(m.fs.RETENTATE.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.6597, rel=1e-3)
assert value(m.fs.separation_factor) == pytest.approx(1038, rel=1e-3)
assert value(m.fs.pervap.heat_duty[0]) == pytest.approx(5813, rel=1e-3)

# 3. Optimization

Suppose we wish to characterize the membrane behavior by calculating the maximum inlet water mole fraction allowing a separation factor of at least 100 (typical value for high-efficiency separation processes such as gas separation of CO2/N2). We need to fix total inlet flow to ensure physically-sound solutions. We can quickly modify and resolve the model, and check some key results:

In [ ]:
# unfix inlet flows but fix total to prevent divergence during solve
m.fs.WATER.outlet.flow_mol_phase_comp[0, "Liq", "water"].unfix()
m.fs.GLYCOL.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"].unfix()
m.fs.total_flow = Constraint(expr=m.fs.WATER.outlet.flow_mol_phase_comp[0, "Liq", "water"] +
                             m.fs.GLYCOL.outlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"] ==
                             1*pyunits.mol/pyunits.s)

# set criteria for separation factor
m.fs.sep_min = Constraint(expr=m.fs.separation_factor >= 100)

# set objective - defaults to minimization
m.fs.obj = Objective(expr=m.fs.inlet_water_frac, sense=maximize)

results = solver.solve(m, tee = True)

In [ ]:
# print results

m.fs.WATER.report()
m.fs.GLYCOL.report()
m.fs.PERMEATE.report()
m.fs.RETENTATE.report()

In [ ]:
print(f"Inlet water mole fraction: {value(m.fs.inlet_water_frac)}")
print(f"Permeate water mole fraction: {value(m.fs.permeate_water_frac)}")
print(f"Separation factor: {value(m.fs.separation_factor)}")
print(f"Condensation duty: {value(m.fs.pervap.heat_duty[0]/1000)} kW")
print(f"Duty per mole water recovered: {value(m.fs.pervap.heat_duty[0]/(1000*m.fs.PERMEATE.inlet.flow_mol_phase_comp[0, 'Liq', 'water']*3600))} kW-h / mol")

In [ ]:
# check results
assert check_optimal_termination(results)
assert_units_consistent(m)

assert value(m.fs.PERMEATE.inlet.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.1426, rel=1e-3)
assert value(m.fs.PERMEATE.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.0002667, rel=1e-3)
assert value(m.fs.RETENTATE.inlet.flow_mol_phase_comp[0, "Liq", "water"]) == pytest.approx(0.6998, rel=1e-3)
assert value(m.fs.RETENTATE.inlet.flow_mol_phase_comp[0, "Liq", "ethylene_glycol"]) == pytest.approx(0.1573, rel=1e-3)
assert value(m.fs.separation_factor) == pytest.approx(100.0, rel=1e-3)
assert value(m.fs.pervap.heat_duty[0]) == pytest.approx(5813, rel=1e-3)

# 4. Summary

The IDAES Skeleton Unit Model is a powerful tool for implementing relatively simple first-princples, surrogate-based or empirical unit operations. More crucially, users can add their own custom models and integrate them into a larger IDAES flowsheet without adding control volumes or rigorous flow balance and equilibrium calculations when not required. The pervaporation example displays a case where all model equations are empirical correlations or simple manual calculations, with a small number of state variable and port connections, and the Skeleton model avoids complex calculations that impact model tractability. The example also demonstrates adding a custom initialization scheme to handle internally model degrees of freedom, a feature providing greater user control than with most IDAES unit models.